### 함수

-----------

### 네이버 접근 함수

In [10]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re

def get_addr(location_name):
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument('window-size=1380,900')
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    
    try:
        search_url = f"https://map.naver.com/p/search/{urllib.parse.quote(location_name)}"
        driver.get(search_url)
        
        # 기본 대기 시간 축소
        WebDriverWait(driver, 3).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        search_list = soup.find(id="_pcmap_list_scroll_container").find_all('li')
        address = search_list[0].find(class_="KgfA6 D7FgR").find(class_="Pb4bU").text
        
        # 주소 추출 로직은 동일하게 유지
        # [기존 코드 유지]
        
    except:
        return "지역 미상"
    finally:
        driver.quit()

## 네이버 검색 함수

In [9]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
import urllib.parse 
import re 
import time 
from selenium.common.exceptions import TimeoutException  

def get_addr(location_name):
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument('window-size=1380,900')
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    
    try:
        search_url = f"https://map.naver.com/p/search/{urllib.parse.quote(location_name)}"
        driver.get(search_url)
        
        # 페이지 로딩 대기 및 iframe 전환
        WebDriverWait(driver, 3).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))
        
        # 검색 결과에서 주소 추출
        soup = BeautifulSoup(driver.page_source, "html.parser")
        search_list = soup.find(id="_pcmap_list_scroll_container").find_all('li')
        address = search_list[0].find(class_="KgfA6 D7FgR").find(class_="Pb4bU").text
        address_parts = address.split()
        
        # 주소에서 행정구역 추출 (서울은 구, 그 외는 시/군)
        if '서울' in address_parts:
            for part in address_parts:
                if part.endswith('구'):
                    return part
            return "서울시"
        else:
            for part in address_parts:
                if part.endswith('시') or part.endswith('군'):
                    return part
            for part in address_parts:
                if part.endswith('도'):
                    return part
            if len(address_parts) > 1:
                return address_parts[1]
                
        return "지역 미상"
        
    except Exception as e:
        print(f"주소 찾기 오류 ({location_name}): {str(e)}")
        return "지역 미상"
    finally:
        driver.quit()

### 구글에서 근처 역가져오기

In [11]:

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re


# 구글 맵에서 검색 후 첫 번째 결과 이름 가져오기
def get_first_result_name(search_query):
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument('window-size=1380,900')
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    
    try:
        driver.get("https://maps.google.com")
        search_box = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "searchboxinput"))
        )
        search_box.clear()
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.ENTER)
        
        # 대기 시간 축소
        first_result = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "Nv2PK"))
        )
        name_element = first_result.find_element(By.CLASS_NAME, "qBF1Pd")
        return name_element.text
    except:
        return None
    finally:
        driver.quit()

### 실행함수

In [12]:
def execute_sql(query, params=None):
    conn = None
    cursor = None
    try:
        conn = mysql.connector.connect(
            host="database-1.cnu82kme6p4d.ap-northeast-2.rds.amazonaws.com",
            port=3306,
            user="eda",
            password="ojk0707",
            database="ojk"
        )
        cursor = conn.cursor()
        cursor.execute(query, params)
        
        if query.strip().upper().startswith("SELECT"):
            return cursor.fetchall()
        else:
            conn.commit()
            return None
    except Exception as e:
        print(f"SQL 오류: {e}")
        return None
    finally:
        if cursor: 
            cursor.close()
        if conn: 
            conn.close()

------------

## DB에 저장하기

In [18]:
##### API 가져와보기
import requests
import pandas as pd 
from urllib.parse import quote
from bs4 import BeautifulSoup
import re
import mysql.connector

# 서울 지역 데이터 가져오기
for i in range(1, 117):
    area_list = []
    # 데이트 장소가 아닌것은 필터링
    valid_numbers = {27, 28, 36, 37, 41, 43, 56}
    if i in valid_numbers:
        continue
    
    area = f"POI{i:03d}"
    api_key = '6e51654f776b353335387168745664'
    url = f'http://openapi.seoul.go.kr:8088/{api_key}/xml/citydata/1/1000/{area}'
    
    
    response = requests.get(url, timeout=10)
    xml_data = response.text
    soup = BeautifulSoup(xml_data, 'lxml-xml')

    # 명소명 처리
    name_region = soup.find('AREA_NM').text.strip()
    locations = [
        '광화문·덕수궁', '국립중앙박물관·용산가족공원', '낙산공원·이화마을', '덕수궁길·정동길', '서리풀공원·몽마르뜨공원',
        '서울식물원·마곡나루역', '신촌·이대역', '오목교역·목동운동장', '종로·청계 관광특구', '창덕궁·종묘', '해방촌·경리단길'
    ]

    if name_region in locations:
        area_nm = name_region.split('·')[-1]  # 뒷부분 선택
    else:
        area_nm = name_region

    # 지하철 역
    try:
        sub = soup.find('SUB_STN_NM')
        station_raw = sub.text + '역'
    except:
        area_nm_search = area_nm.replace("·", "") + ' 근처 역'
        station_raw = get_first_result_name(area_nm_search)
        if station_raw:
            # 괄호와 내용 제거 (먼저 실행)
            station_raw = re.sub(r"\(.*?\)", "", station_raw)

            # 문제 역 교정
            if "4.19민주묘지" in station_raw:
                station_raw = "수유역"
            elif "응암순환" in station_raw:
                station_raw = "응암역"
            elif "대공원고가교" in station_raw:
                station_raw = "청계산입구역"

            # "역"이 포함된 경우, "역" 뒤 제거
            if "역" in station_raw:
                station_raw = re.sub(r"(역).", r"\1", station_raw)
            # "역"이 없는 경우, "역" 추가
            else:
                station_raw = station_raw + "역"
        else:
            station_raw = "역 정보 없음"
    # API 데이터에도 괄호 제거 적용 (일관성 유지)
    station_raw = re.sub(r"\(.?\)", "", station_raw)

    # datetime
    update_time = soup.find('PPLTN_TIME').text
    #최대 인구
    max_pepl = int(soup.find('AREA_PPLTN_MAX').text)
    #최소 인구
    min_pepl = int(soup.find('AREA_PPLTN_MIN').text)
    #기온
    temp = float(soup.find('TEMP').text)
    #강수
    rain_mm = soup.find('PRECIPITATION').text
    if rain_mm == "-":
        rain_mm = 0
    else:
        rain_mm_match = re.search(r'\d+', rain_mm)  # 숫자만 찾기
        rain_mm = int(rain_mm_match.group()) if rain_mm_match else 0
    #강수확률
    rain_per = int(soup.find('RAIN_CHANCE').text)
    #강수형태
    rain_type = soup.find('PRECPT_TYPE').text
    #대기질
    air_type = soup.find('AIR_IDX_MVL').text
    #하늘상태
    sky_stts = soup.find('SKY_STTS').text
    #습도
    air_water = soup.find('HUMIDITY').text
    #풍향
    wind_level = float(soup.find('WIND_SPD').text)
    #자외선 지수
    uv = soup.find('AIR_IDX_MVL').text.strip()
    uv_index = 0 if uv == '점검중' else float(uv)

    try:
        address_element = soup.find('ADDRESS')
        if address_element:
            address = address_element.text
            match = re.search(r'[가-힣]+구', address)
            if match:
                gu = match.group()
            else:
                gu = get_addr(area_nm)
        else:
            gu = get_addr(area_nm)
    except:
        gu = get_addr(area_nm)

    # 구 정보가 없으면 기본값 설정
    if not gu:
        gu = "지역 미상"
    area_list += [update_time, area_nm, station_raw, gu, min_pepl, max_pepl, temp, rain_mm, rain_per, rain_type, air_type, sky_stts, air_water, wind_level, uv_index]

    query = """INSERT INTO spot (datetime, region, station, gu, min_population, max_population, temp, precipitation,
    rain_chance, precpt_type, air_idx, sky_stts, humidity, wind_spd, uv_idx) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

    execute_sql(query, area_list)

    print(update_time, area_nm, station_raw, gu, min_pepl, max_pepl, temp, rain_mm, rain_per, rain_type, air_type, sky_stts, air_water, wind_level, uv_index)

2025-03-28 07:15 강남 MICE 관광특구 삼성역 강남구 3000 3500 4.8 0 0 없음 90.0 구름많음 51 3.6 90.0
2025-03-28 07:15 동대문 관광특구 동대문역 중구 12000 14000 3.7 0 0 없음 97.0 구름많음 59 3.5 97.0
2025-03-28 07:15 명동 관광특구 시청역 중구 14000 16000 3.7 0 0 없음 92.0 구름많음 59 3.5 92.0
2025-03-28 07:15 이태원 관광특구 녹사평역 용산구 4000 4500 4.6 0 0 없음 87.0 구름많음 52 3.0 87.0
2025-03-28 07:15 잠실 관광특구 잠실역 송파구 22000 24000 4.9 0 0 없음 92.0 구름많음 50 2.7 92.0
2025-03-28 07:15 청계 관광특구 동대문역 종로구 12000 14000 3.7 0 0 없음 97.0 구름많음 59 3.5 97.0
2025-03-28 07:15 홍대 관광특구 합정역 마포구 20000 22000 4.7 0 0 없음 85.0 구름많음 52 4.1 85.0
2025-03-28 07:15 경복궁 경복궁역 종로구 500 600 3.7 0 0 없음 97.0 구름많음 59 3.5 97.0
2025-03-28 07:15 덕수궁 시청역 중구 10000 12000 3.7 0 0 없음 97.0 구름많음 59 3.5 97.0
2025-03-28 07:15 보신각 종각역 종로구 3000 3500 3.7 0 0 없음 97.0 구름많음 59 3.5 97.0
2025-03-28 07:15 서울 암사동 유적 암사역공원역번출구 강동구 400 500 4.9 0 0 없음 100.0 구름많음 50 2.7 100.0
2025-03-28 07:20 종묘 종로3가역 종로구 5000 5500 3.7 0 0 없음 97.0 구름많음 59 3.5 97.0
2025-03-28 07:20 가산디지털단지역 가산디지털단지역 금천구 14000 16000 4.8 0 0 없음 97.0 구름많음 49 2.